THIS FILE SHOULD BE RUN ON YOUR LOCAL(JUPYTER NOTEBOOK)

Extracting all text by using selenium library

In [1]:
#Importing packages
from selenium import webdriver
import pandas as pd

You need to download chromedriver then copy-paste chromedriver.exe path to the line below

In [2]:
driver = webdriver.Chrome(r"C:\Users\MONSTER\Desktop\driver\chromedriver.exe")

In [3]:
driver.get('https://huubsh.com/l/tv-plus-2963')

In [1]:
#//div[contains(@class,'description')]---> Xpath

In [5]:
text = driver.find_elements_by_xpath("//div[contains(@class,'description')]")[0]
all_text = text.text
all_text

'TV for today. 100% freedom. 0% risk. Connect. Enjoy!\nWhy pay too much for your TV and Internet? Order TV PLUS now and really save!\n160 TV channels in the basic subscription, 75+ in HD. 7-day Replay. Live Pause. Up to 1000h recording. 10’000+ Radio stations & Podcasts. Video on Demand with 5000+ Movies & TV Shows. Access to YouTube, Netflix, Sky.\n0% risk: you can cancel monthly at no cost after the first 3 months!'

import NLP libraries

In [6]:
#from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download("punkt")
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from warnings import filterwarnings
filterwarnings('ignore')
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MONSTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


lenght of our all_text

In [7]:
len(all_text)

416

tokenize word

In [8]:
text=word_tokenize(all_text.lower())

only alphabetical strings

In [9]:
text = [w for w in text if w.isalpha()]

In [10]:
stop_words = stopwords.words('english')

meaningful words that come from stopwords

In [11]:
text = [t for t in text if t not in stop_words]

meaningful text lenght

In [12]:
len(text)

42

Word embedding

In [13]:
import numpy as np

In [14]:
from tensorflow.keras.models import Sequential,Model

In [15]:
from tensorflow.keras.layers import Embedding

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
MAX_NB_WORDS = 100


In [18]:
text[:2]

['tv', 'today']

modelling

In [19]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 37 unique tokens.


In [20]:
word_index

{'tv': 1,
 'risk': 2,
 'today': 3,
 'freedom': 4,
 'connect': 5,
 'enjoy': 6,
 'pay': 7,
 'much': 8,
 'internet': 9,
 'order': 10,
 'plus': 11,
 'really': 12,
 'save': 13,
 'channels': 14,
 'basic': 15,
 'subscription': 16,
 'hd': 17,
 'replay': 18,
 'live': 19,
 'pause': 20,
 'recording': 21,
 'radio': 22,
 'stations': 23,
 'podcasts': 24,
 'video': 25,
 'demand': 26,
 'movies': 27,
 'shows': 28,
 'access': 29,
 'youtube': 30,
 'netflix': 31,
 'sky': 32,
 'cancel': 33,
 'monthly': 34,
 'cost': 35,
 'first': 36,
 'months': 37}

 load the whole embedding into memory, I used glove trained model

In [21]:
#glove has 100 dimension and 6B words
embeddings_index = dict()
f = open('./glove.6B.100d.txt', encoding="utf-8")
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [22]:
len(embeddings_index['tv'])

100

In [23]:
len(embeddings_index)

400000

In [24]:
#embeddings_index

Adding 1 because of reserved 0 index. Later we will reduce this zero.

In [25]:

vocab_size = len(tokenizer.word_index) + 1
print (vocab_size)

38


create a weight matrix for words in training docs

In [26]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [27]:
tokenizer.word_index.get("tv")

1

In [28]:
tokenizer.word_index.get("enjoy")

6

In [29]:
embedding_matrix.shape

(38, 100)

tv embedding_matrix 

In [30]:
embedding_matrix[1]

array([-1.82319999e-01, -6.51359975e-01,  2.85890013e-01, -5.84060013e-01,
       -6.85850009e-02,  1.81869995e-02,  1.07560001e-04,  4.02649999e-01,
        1.18539996e-01, -1.27820000e-01,  2.29809999e-01, -4.30880010e-01,
       -6.67579994e-02,  1.75310001e-01, -1.78210009e-02, -3.04679990e-01,
        2.58060008e-01, -3.22649986e-01,  5.87409973e-01,  1.08410001e+00,
       -2.52739996e-01,  1.37869999e-01,  2.27320001e-01, -2.30910003e-01,
        9.66520011e-01, -1.23489998e-01,  1.10060000e+00,  6.48649991e-01,
       -4.40139994e-02,  3.71639997e-01,  4.49270010e-01,  1.03509998e+00,
       -1.03119999e-01,  5.73920012e-01,  5.15359998e-01,  2.09240004e-01,
       -4.79530007e-01, -8.22210014e-02, -2.57900000e-01, -7.89790004e-02,
        2.98400000e-02,  3.88090014e-01, -1.12639999e+00,  3.76670003e-01,
        2.14419998e-02, -4.48430002e-01, -7.08310008e-01, -5.22840023e-01,
        2.40060002e-01, -5.74129999e-01, -6.85920000e-01,  3.57969999e-01,
        4.79420006e-01,  

In order to find similarity of words, using cosine_similarity

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [32]:
sim_mat = cosine_similarity(embedding_matrix,embedding_matrix)

similarity matrix for tv

In [33]:
sim_mat[1]

array([0.        , 1.        , 0.27940192, 0.51553629, 0.29712011,
       0.34737541, 0.36570946, 0.4260918 , 0.4686075 , 0.60545597,
       0.33474674, 0.48850552, 0.45750282, 0.32059851, 0.69355068,
       0.31465459, 0.42601377, 0.55194836, 0.46281192, 0.6234564 ,
       0.20348052, 0.44817797, 0.78830825, 0.61368683, 0.36937906,
       0.74173718, 0.32197855, 0.71559458, 0.72408647, 0.42250074,
       0.55416704, 0.30516475, 0.57257811, 0.28748322, 0.36629492,
       0.3838893 , 0.50037266, 0.3895824 ])

In [34]:
#We should get rid of first item which is zero. The rest of them are similarity with "tv"
sim_mat[1][1:]

array([1.        , 0.27940192, 0.51553629, 0.29712011, 0.34737541,
       0.36570946, 0.4260918 , 0.4686075 , 0.60545597, 0.33474674,
       0.48850552, 0.45750282, 0.32059851, 0.69355068, 0.31465459,
       0.42601377, 0.55194836, 0.46281192, 0.6234564 , 0.20348052,
       0.44817797, 0.78830825, 0.61368683, 0.36937906, 0.74173718,
       0.32197855, 0.71559458, 0.72408647, 0.42250074, 0.55416704,
       0.30516475, 0.57257811, 0.28748322, 0.36629492, 0.3838893 ,
       0.50037266, 0.3895824 ])

In [35]:
for key, value in word_index.items() :
    print (key,value)

tv 1
risk 2
today 3
freedom 4
connect 5
enjoy 6
pay 7
much 8
internet 9
order 10
plus 11
really 12
save 13
channels 14
basic 15
subscription 16
hd 17
replay 18
live 19
pause 20
recording 21
radio 22
stations 23
podcasts 24
video 25
demand 26
movies 27
shows 28
access 29
youtube 30
netflix 31
sky 32
cancel 33
monthly 34
cost 35
first 36
months 37


Create Data Frame in pandas

In [36]:
df = pd.DataFrame.from_dict(word_index, orient='index')
df=df.rename(columns={0: "similarity"})

Now similarity values come from index numbers

In [37]:
df.head()

,similarity
tv,1
risk,2
today,3
freedom,4
connect,5


implementing similarity index for tv that's index number equal to 1 

In [60]:
df.similarity=sim_mat[1][1:]

In [61]:
df.head()

,similarity
tv,1.000000
risk,0.279402
today,0.515536
freedom,0.297120
connect,0.347375


top 30 similarity for tv

In [63]:
df.sort_values('similarity', ascending = False)[:30]

,similarity
tv,1.000000
radio,0.788308
video,0.741737
shows,0.724086
movies,0.715595
channels,0.693551
live,0.623456
stations,0.613687
internet,0.605456
sky,0.572578


As we see above, subscription similarity is 0.426014 for tv

In [46]:
df.similarity = sim_mat[22][1:]

top 10 similarity for radio

In [47]:
df.sort_values('similarity', ascending = False)[:10]

,similarity
radio,1.000000
tv,0.788308
stations,0.718944
channels,0.626238
video,0.602876
live,0.570291
shows,0.559567
recording,0.545348
internet,0.506790
sky,0.492584


similarity for subscription index number is 16

In [64]:
df.similarity=sim_mat[16][1:]

top 10 similarity for subscription

In [65]:
df.sort_values('similarity', ascending = False)[:10]

,similarity
subscription,1.000000
monthly,0.621883
internet,0.583962
netflix,0.553527
channels,0.502108
pay,0.498126
access,0.483084
podcasts,0.479430
youtube,0.476813
cost,0.460982


by calling word_sim function that is below, you can easily find any word similarity

In [50]:
def word_sim(word):
    index = tokenizer.word_index.get(word)
    df = pd.DataFrame.from_dict(word_index, orient='index')
    df=df.rename(columns={0: "similarity"})
    df.similarity=sim_mat[index][1:]
    df = df.sort_values('similarity', ascending = False)[:10]
    return df

pick a word

In [57]:
word_index.keys()

dict_keys(['tv', 'risk', 'today', 'freedom', 'connect', 'enjoy', 'pay', 'much', 'internet', 'order', 'plus', 'really', 'save', 'channels', 'basic', 'subscription', 'hd', 'replay', 'live', 'pause', 'recording', 'radio', 'stations', 'podcasts', 'video', 'demand', 'movies', 'shows', 'access', 'youtube', 'netflix', 'sky', 'cancel', 'monthly', 'cost', 'first', 'months'])

run word_sim function within 'pay'

In [51]:
word_sim('pay')

,similarity
pay,1.000000
cost,0.769212
plus,0.641230
much,0.625675
save,0.582991
monthly,0.581118
months,0.571993
cancel,0.568468
order,0.567203
demand,0.555931


In [58]:
word_sim('movies')

,similarity
movies,1.000000
tv,0.715595
shows,0.684054
video,0.665285
live,0.582449
really,0.518347
much,0.505095
channels,0.502623
internet,0.501160
today,0.477755
